# Importar librerías

In [1]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
import albumentations as alb
import cv2
import os
import time
import uuid

C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Creación de modelo

### Importación de capas

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

### Descarga de VGG16

Se descarga la red neuronal convolucional VGG16 ya preentrenada, sin las capas finales.

In [3]:
vgg = VGG16(include_top=False)

### Construir el modelo

El modelo consiste en dos partes:
   - Clasificación: Para determinar la etiqueta de la detección.
   - Regresión: Para predecir las coordenadas del rectángulo donde se encuentra el objeto.
   
Cada parte se basa en las *feature* entregadas por la red VGG16 y agrega capas sobre esta.
   - Clasificación: Agrega una capa de *Max Pooling*, luego una capa oculta *fully connected* y finalmente la capa de salida con una neurona y activación sigmoidea.
   - Regresión: Agrega una capa de *Max Pooling*, luego una capa oculta *fully connected* y finalmente la capa de salida con 4 neuronas de salida y función de activación sigmoidea.
   
   


In [4]:
def build_model(): 
    capa_entrada = Input(shape=(120,120,3))
    
    vgg = VGG16(include_top=False)(capa_entrada)

    # Modelo para clasificación
    capa_pooling_clasificador = GlobalMaxPooling2D()(vgg)
    capa_oculta_clasificador = Dense(2048, activation='relu')(capa_pooling_clasificador)
    salida_clasificador = Dense(1, activation='sigmoid')(capa_oculta_clasificador)
    
    # Modelo para predicción de coordenadas del rectángulo
    capa_pooling_regresor = GlobalMaxPooling2D()(vgg)
    capa_oculta_regresor = Dense(2048, activation='relu')(capa_pooling_regresor)
    salida_regresor = Dense(4, activation='sigmoid')(capa_oculta_regresor)
    
    modelo = Model(inputs=capa_entrada, outputs=[salida_clasificador, salida_regresor])
    return modelo

In [5]:
modelo = build_model()

### Crear clase Modelo personalizada

Requiere definir los siguientes métodos:

- **compile**: Para declarar las funciones de *loss* y el optimizador a utilizar para entrenar.
- **train_step**: Para ajustar los pesos del modelo usando gradiente descendiente. Notar que se usa como función de *loss* una suma ponderada del *loss* de regresión y el de clasificación.
- **test_step**: Para realizar una predicciones sobre el conjunto de *test*. Guarda las *losses* en un diccionario.
- **call**: Para hacer predicciones con el modelo, se llama con el método *predict*.

In [6]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, optimizador, loss_clasificacion, loss_regresion, **kwargs):
        super().compile(**kwargs)
        self.loss_clasificacion = loss_clasificacion
        self.loss_regresion = loss_regresion
        self.opt = optimizador
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            clases, coords = self.model(X, training=True)
            
            loss_clasificacion_batch = self.loss_clasificacion(y[0], clases)
            loss_regresion_batch = self.loss_regresion(tf.cast(y[1], tf.float32), coords)
            
            loss_total = loss_regresion_batch+0.5*loss_clasificacion_batch
            
            grad = tape.gradient(loss_total, self.model.trainable_variables)
        
        self.opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"loss_total":loss_total, "loss_clasificacion":loss_clasificacion_batch, "loss_regresion":loss_regresion_batch}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        clases, coords = self.model(X, training=False)
        
        loss_clasificacion_batch = self.closs(y[0], clases)
        loss_regresion_batch = self.lloss(tf.cast(y[1], tf.float32), coords)
        loss_total = loss_regresion_batch +0.5*loss_clasificacion_batch
        
        {"loss_total":loss_total, "loss_clasificacion":loss_clasificacion_batch, "loss_regresion":loss_regresion_batch}
    
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [7]:
modelo = FaceTracker(modelo)

### Cargar el modelo

In [8]:
from tensorflow.keras.models import load_model
modelo = load_model('modelo.h5')

# Implementar el modelo

- Se abre videocamera con OpenCV.
- Se toman los *frame* y se recortan, tomando los primeros *450 x 450* pixeles.
- Se reescala y normaliza el *frame* para poder ingresarlo al modelo.
- Se realiza predicción con el modelo.
- Se dibuja rectangulo con cv2.rectangle y label con cv2.putText.
- Se muestra el *frame* modificado en pantalla.
- Salir presionando "q".

In [15]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    
    tamano_frame = [450, 450]
    print(frame.shape)
    frame = frame[0:450, 0:450,:]
    
    print(frame.shape)
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_escalada = tf.image.resize(rgb, (120,120))
    
    prediccion = modelo.predict(np.expand_dims(img_escalada/255,0))
    coords = prediccion[1][0]
    
    if prediccion[0] > 0.5: 
        # Recuadro de prediccion
        cv2.rectangle(frame, 
                      tuple(np.multiply(coords[:2], tamano_frame).astype(int)),
                      tuple(np.multiply(coords[2:], tamano_frame).astype(int)),
                            (255,0,0), 2)
        # Recuadro para label
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(coords[:2], tamano_frame).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(coords[:2], tamano_frame).astype(int),
                                    [85,0])), 
                            (255,0,0), -1)
        
        # Texto con label de clasificación
        cv2.putText(frame, 'rostro', tuple(np.add(np.multiply(coords[:2], tamano_frame).astype(int),
                                               [5,-8])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('FaceTracker', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 60ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 59ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [=======

1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 46ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 56ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 77ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 61ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 54ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0

1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 53ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 56ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 54ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 54ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 45ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 55ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0

1/1 [==============================] - 0s 58ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 57ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 61ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 55ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 53ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 55ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0

1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 55ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 46ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 55ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 54ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0

1/1 [==============================] - 0s 53ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 55ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 56ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 54ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 54ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 57ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 53ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 59ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0

1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0

1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 46ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 51ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 54ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0

1/1 [==============================] - 0s 53ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 46ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 49ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 53ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0

1/1 [==============================] - 0s 54ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 47ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 52ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 50ms/step
(480, 640, 3)
(450, 450, 3)
1/1 [==============================] - 0s 48ms/step
